# LLaMA 3.2 3B 모델 프롬프트 튜닝 실습

In [ ]:
# 환경 구성
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델 준비
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# 추론 함수 제작
def inference(messages):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

In [ ]:
question = "프롬프트 튜닝에 대해 알려줘"
messages = [
    {"role": "user", "content": f"{question}"}
    ]
answer = inference(messages)
print(answer)

In [ ]:
question = "프롬프트 튜닝에 대해 알려줘"
messages = [
    {"role": "system", "content": "너는 친절한 한국어 챗봇이야. 질문에 대해 정리된 답을 알려줘"},
    {"role": "user", "content": f"{question}"}
    ]
answer = inference(messages)
print(answer)